In [3]:
import pandas as pd
import os
path = '/Users/simonrahenbrock/Documents/thesis_code/datasets/'


In [75]:
path = '/Users/simonrahenbrock/Documents/thesis_code/datasets/archive'
df_lu_weather = pd.read_csv(f'{path}ludhiana 2022-09-01 to 2023-06-30.csv')
df_lu_history = pd.read_csv(f'{path}ludhiana 2017-02-04 to 2019-03-31.csv')
df_lu_poll = pd.read_csv(f'{path}Air Pollutant at 400pm PM25 Punjab Ludhiana Average.csv')
df_na_weather = pd.read_csv(f'{path}nairobi 2022-01-01 to 2023-06-30.csv')
df_pm25_na = pd.read_csv(f'{path}pm25_na_extended.csv')
df_pm10_na = pd.read_csv(f'{path}pm10_na_extended.csv')
df_lu_vehicles = pd.read_excel(f'{path}Registered Motor Vehicles Punjab Fuel Type Petrol.xlsx')
df_lu_rice_ci = pd.read_csv(f'{path}Average Consumer Prices Industrial Workers Punjab Ludhiana Rice.csv')
df_pm25_na_embassy = pd.read_csv(f'{path}nairobi-us-embassy-air-quality.csv')

In [16]:
df_pm25_na_embassy = pd.read_csv(f'{path}nairobi-us-embassy-air-quality.csv')

In [866]:
df_kenya_food_ci = pd.read_excel(f'{path}CPI_Food_and_Non_Alcoholic_Beverages_Kenya_distributed.xlsx')
df_kenya_alc_ci = pd.read_excel(f'{path}CPI_Alcoholic_Beverages_Tobacco_and_Narcotics_Kenya_distributed.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/simonrahenbrock/Documents/thesis_code/datasets/archive/CPI_Food_and_Non_Alcoholic_Beverages_Kenya_distributed.xlsx'

In [868]:
df_india_alc_ci_distributed = pd.read_csv(f'{path}CPI_PanTobacco_and_Intoxicants_India_distributed.csv')
#df_india_food_ci_distributed = pd.read_csv(f'{path}/archive/CPI_Food_and_Beverages_India_distributed.csv')

In [175]:
df_na_weather = pd.read_csv(f'{path}nairobi 2022-01-01 to 2023-06-30.csv')


,date,CPI_PanTobacco_and_Intoxicants_India_distributed
0,2011-01-03,16.48
1,2011-01-10,16.48
2,2011-01-17,16.48
3,2011-01-24,16.48
4,2011-01-31,16.48
...,...,...
642,2023-04-24,50.15
643,2023-05-01,40.20
644,2023-05-08,40.20
645,2023-05-15,40.20


# Preprocess Data From CEC

In [990]:
def preprocess_cec(df_old, new_col_name):
    df = df_old.copy()
    df = df.loc[26:]
    # Convert the 'Year' column to datetime and set it as the index
    df['date'] = pd.to_datetime(df['Unnamed: 0'])
    df.set_index('date', inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    # Rename the column name to a valid Python identifier
    df.rename(columns={df.columns[0]: new_col_name}, inplace=True)
    df[new_col_name] = df[new_col_name].astype(float)
    df.to_csv(f'{path + new_col_name}.csv')
    return df



In [ ]:
df_india_alc_ci_distributed = preprocess_cec(df_india_alc_ci_distributed, 'CPI_PanTobacco_and_Intoxicants_India_distributed')
df_india_food_ci_distributed = preprocess_cec(df_india_food_ci_distributed, 'CPI_Food_and_Beverages_India_distributed')


# Consumer Prices Index India - Monthly to Weekly

In [771]:
cpi_health = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Health_YoY.csv')
cpi_trans = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Transport and Communication_YoY.csv')
cpi_tobac = pd.read_csv(f'{path}Consumer Price Index Punjab Pan Tobacco and Intoxicants_YoY.csv')
cpi_health_cubic = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Health_cubic_interpolation.csv')
cpi_total = pd.read_csv(f'{path}Consumer Price Index Punjab_cubic_interpolated.csv')
cpi_tobac_pop_cub = pd.read_csv(f'{path}Consumer Price Index Punjab Pan Tobacco and Intoxicants_cub_PoP.csv')


In [ ]:
cpi_absv = pd.read_csv(f'{path}Punjab_CPI_absolutevalues.csv')
cpi_food_absv = pd.read_csv(f'{path}Punjab_CPI_Food_and_Beverages_absolutevalues.csv')

In [897]:
def preprocess_cpi(df, col_name):
    # preprocess dataframe
    df = preprocess_cec(df, col_name)
    # resample to a sunday
    df = df.resample('W-SUN').ffill()
    # shift by 4 weeks to avoid data leakage
    df[col_name] = df[col_name].shift(4)
    
    return df

In [918]:
cpi_total_interpolated = pd.read_csv(f'{path}Consumer Price Index Punjab_cubic_interpolated.csv')
cpi_health_absv_interpolated = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Health_cubic_interpolation.csv')
cpi_health_pop_cub = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Health_PoP_cubic_interpol.csv')
cpi_health_yoy = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Health_YoY.csv') 
cpi_health_absv_distributed = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Health_absv_distributed.csv')
cpi_health_pop_distributed = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Health_PoP_distributed.csv')
cpi_food_interpolated = pd.read_csv(f'{path}Punjab_CPI_Food_and_Beverages_absolutevalues.csv')
cpi_tobac_interpolated = pd.read_csv(f'{path}Consumer Price Index Punjab Pan Tobacco and Intoxicants_cub_PoP.csv')



In [919]:
def preprocess_and_merge(dfs, col_names):
    processed_dfs = []
    
    result_df = None
    for df, col_name in dfs:
        df = preprocess_cpi(df, col_name)
        if result_df is None:
            result_df = df
        else:
            result_df = result_df.join(df)
    return result_df

dfs = [
    (cpi_total_interpolated, 'cpi_total_interpolated'),
    (cpi_health_absv_interpolated, 'cpi_health_interpolated'),
    (cpi_health_pop_cub, 'cpi_health_PoP_interpolated'),
    (cpi_health_yoy, 'cpi_health_YoY'),
    (cpi_health_absv_distributed, 'cpi_health_distributed'),
    (cpi_health_pop_distributed, 'cpi_health_PoP_distributed'),
    (cpi_food_interpolated, 'cpi_food_cubic_interpolated'),
    (cpi_tobac_interpolated, 'cpi_tobac_interpolated')
]


merged_df = preprocess_and_merge(dfs, col_names)


/var/folders/_t/kmv70h2x57sbxhd8w5ybzqwr0000gn/T/ipykernel_8701/4202089750.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['Unnamed: 0'])


In [920]:
merged_df.loc['2013-04-02':]

,cpi_total_interpolated,cpi_health_interpolated,cpi_health_PoP_interpolated,cpi_health_YoY,cpi_health_distributed,cpi_health_PoP_distributed,cpi_food_cubic_interpolated,cpi_tobac_interpolated
date,,,,,,,,
2013-04-07,105.400000,103.700000,0.000000,NaN,25.925,0.000000,106.900000,0.478469
2013-04-14,105.622234,103.834131,0.129508,NaN,25.925,0.000000,107.050769,1.750855
2013-04-21,105.879713,104.035713,0.341221,NaN,25.925,0.000000,107.253137,3.258723
2013-04-28,106.122335,104.269438,0.575188,NaN,25.925,0.000000,107.478936,4.637799
2013-05-05,106.300000,104.500000,0.771456,NaN,20.900,0.154291,107.700000,5.523810
...,...,...,...,...,...,...,...,...
2023-05-07,171.400000,183.500000,0.658256,5.946882,45.875,0.164564,177.900000,2.493075
2023-05-14,171.536341,183.684671,0.615531,NaN,45.875,0.164564,178.115351,2.547791
2023-05-21,171.618021,183.804558,0.505073,NaN,45.875,0.164564,178.356220,2.372774


In [921]:
merged_df.drop('cpi_health_YoY', axis=1, inplace=True)

In [842]:
cpi_tobac_pop_cub = pd.read_csv(f'{path}Consumer Price Index Punjab Pan Tobacco and Intoxicants_cub_PoP.csv')
cpi_tobac_pop_cub = preprocess_cec(cpi_tobac_pop_cub, 'cpi_tobac_cub_PoP')
cpi_tobac_pop_cub = cpi_tobac_pop_cub.resample('W-SUN').ffill()

In [824]:
cpi_health_pop_cub = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Health_PoP_cubic_interpol.csv')

In [790]:
cpi_total = pd.read_csv(f'{path}Consumer Price Index Punjab_cubic_interpolated.csv')
cpi_total = preprocess_cec(cpi_total, 'cpi_total')
cpi_health_pop_cub = preprocess_cec(cpi_health_pop_cub, 'cpi_health_cub_PoP')

In [821]:
cpi_total = cpi_total.resample('W-SUN').ffill()

In [772]:
df_cpi_health = preprocess_cec(df_cpi_health, 'cpi_health_YoY')
df_cpi_trans = preprocess_cec(df_cpi_trans, 'cpi_transport_YoY')
df_cpi_tobac = preprocess_cec(df_cpi_tobac, 'cpi_tobac_YoY')
df_cpi_health_cubic = preprocess_cec(df_cpi_health_cubic, 'cpi_health_cub')

/var/folders/_t/kmv70h2x57sbxhd8w5ybzqwr0000gn/T/ipykernel_8701/4202089750.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['Unnamed: 0'])
/var/folders/_t/kmv70h2x57sbxhd8w5ybzqwr0000gn/T/ipykernel_8701/4202089750.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['Unnamed: 0'])
/var/folders/_t/kmv70h2x57sbxhd8w5ybzqwr0000gn/T/ipykernel_8701/4202089750.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['Unnamed: 0'])


In [773]:
df_cpi_health = df_cpi_health.resample('W-SUN').ffill()
df['cpi_health'] = df['cpi_health'].shift(4)

df_cpi_tobac = df_cpi_tobac.resample('W-SUN').ffill()
df_cpi_trans = df_cpi_trans.resample('W-SUN').ffill()
df_cpi_health_cubic = df_cpi_health_cubic.resample('W-SUN').ffill()
cpi_health_pop_cub = cpi_health_pop_cub.resample('W-SUN').ffill()

In [850]:
df_cpi_health_cubic.shift(4)
cpi_total_cubic.shift(4)
df_cpi_tobac

NameError: name 'cpi_total_cubic' is not defined

In [855]:
cpi_health_pop_cub

,cpi_tobac_cub_PoP
date,
2011-02-07,-0.361011
2011-02-14,-0.865703
2011-02-21,-1.144495
2011-02-28,-0.971485
2011-03-07,-0.120773
...,...
2023-05-01,1.729730
2023-05-08,1.441043
2023-05-15,1.206284


In [774]:
df_cpi_ludhiana = df_cpi_health.merge(df_cpi_tobac, left_index=True, right_index=True)
df_cpi_ludhiana = df_cpi_ludhiana.merge(df_cpi_trans, left_index=True, right_index=True)
df_cpi_ludhiana = df_cpi_ludhiana.merge(df_cpi_health_cubic, left_index=True, right_index=True)

In [840]:
df_ludhiana_trainset = pd.read_csv(f'{path}final_dataset_HA_Ludhiana_2017-2019.csv')
df_ludhiana_trainset['date'] = pd.to_datetime(df_ludhiana_trainset['date'])
df_ludhiana_trainset.set_index('date', inplace=True)
#df_ludhiana_trainset.drop('Unnamed: 0', axis=1, inplace=True)

In [818]:
df_ludhiana_trainset = df_ludhiana_trainset.merge(df_cpi_health_cubic, left_index=True, right_index=True)
df_ludhiana_trainset = df_ludhiana_trainset.merge(cpi_total, left_index=True, right_index=True)
df_ludhiana_trainset = df_ludhiana_trainset.merge(cpi_health_pop_cub, left_index=True, right_index=True)

In [843]:
df_ludhiana_trainset = df_ludhiana_trainset.merge(cpi_tobac_pop_cub, left_index=True, right_index=True)

In [930]:
df_ludhiana_trainset_.columns

Index(['tempmax', 'Outcome_EXPIRY_sum', 'Outcome_DAMA_sum',
       'Outcome_DISCHARGE_sum', 'SMOKING__sum', 'ALCOHOL_sum', 'DM_sum',
       'HTN_sum', 'PRIOR_CMP_sum', 'readmissions_sum', 'CKD_sum',
       'SEVERE_ANAEMIA_sum', 'ANAEMIA_sum', 'STABLE_ANGINA_sum', 'ACS_sum',
       'STEMI_sum', 'ATYPICAL_CHEST_PAIN_sum', 'HEART_FAILURE_sum',
       'HFREF_sum', 'HFNEF_sum', 'VALVULAR_sum', 'CAD_sum', 'total_admissions',
       'condition_rainy', 'TYPE_OF_ADMISSION_EMERGENCY_OPD_sum', 'temp', 'dew',
       'humidity', 'windspeed', 'winddir', 'sealevelpressure', 'PM10 AVG',
       'PM2.5 AVG', 'visibility', 'TLC_median', 'PLATELETS_median',
       'GLUCOSE_median', 'UREA_median', 'CREATININE_median', 'BNP_median',
       'solarenergy', 'uvindex', 'tempmin', 'sun_duration', 'cloudcover',
       'male_ratio', 'rural_ratio', 'EF_median', 'HB_mean',
       'DURATION_OF_STAY_median', 'duration_of_intensive_unit_stay_median',
       'average_age', 'search_HeroDMCHeartInstitute', 'search_chestpa

In [927]:
df_ludhiana_trainset_ = df_ludhiana_trainset.merge(merged_df, left_index=True, right_index=True)

In [929]:
merged_df

,cpi_total_interpolated,cpi_health_interpolated,cpi_health_PoP_interpolated,cpi_health_distributed,cpi_health_PoP_distributed,cpi_food_cubic_interpolated,cpi_tobac_interpolated
date,,,,,,,
2011-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-02-06,90.800000,NaN,NaN,NaN,NaN,92.300000,NaN
...,...,...,...,...,...,...,...
2023-05-07,171.400000,183.500000,0.658256,45.875,0.164564,177.900000,2.493075
2023-05-14,171.536341,183.684671,0.615531,45.875,0.164564,178.115351,2.547791
2023-05-21,171.618021,183.804558,0.505073,45.875,0.164564,178.356220,2.372774


In [937]:
"""
1. ) health POP distributed
2. ) food and beverages pop distributed
3. ) health YoY distributed
4. ) registered motor vehicles PoP weekly(original weekly data)
"""

cpi_health_pop_distributed = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Health_PoP_distributed.csv')
cpi_FaB_pop_distributed = pd.read_csv(f'{path}Consumer Price Index Punjab Food and Beverages_PoP_distributed.csv')
cpi_health_yoy_distributed = pd.read_csv(f'{path}Consumer Price Index Punjab Miscellaneous Health_YoY_distributed.csv')
vehicles_units_pop = pd.read_csv(f'{path}Registered Motor Vehicles Punjab_PoP_distributed.csv')

dfs = [
    (cpi_health_pop_distributed, 'cpi_health_PoP'),
    (cpi_FaB_pop_distributed, 'cpi_Food_Beverages_PoP'),
    (cpi_health_yoy_distributed, 'cpi_health_YoY'),
    (vehicles_units_pop, 'vehicles_units_PoP')
]

merged_df = preprocess_and_merge(dfs, col_names)


In [938]:
merged_df

,cpi_health_PoP,cpi_Food_Beverages_PoP,cpi_health_YoY,vehicles_units_PoP
date,,,,
2013-02-10,NaN,0.167946,NaN,NaN
2013-02-17,NaN,0.167946,NaN,NaN
2013-02-24,NaN,0.167946,NaN,NaN
2013-03-03,NaN,0.167946,NaN,NaN
2013-03-10,0.121124,0.166826,NaN,NaN
...,...,...,...,...
2023-05-28,0.164564,0.131349,5.946882,1.646447
2023-06-04,0.054496,-0.081301,5.868815,5.967604
2023-06-11,0.054496,-0.081301,5.868815,-0.741933


In [1101]:
df_ludhiana_trainset = df_ludhiana_trainset.merge(merged_df, left_index=True, right_index=True)

In [1102]:
df_ludhiana_trainset

,tempmax,Outcome_EXPIRY_sum,Outcome_DAMA_sum,Outcome_DISCHARGE_sum,SMOKING__sum,ALCOHOL_sum,DM_sum,HTN_sum,PRIOR_CMP_sum,readmissions_sum,...,search_chestpain,Registered_Motor_Vehicles,cpi_health_PoP_x,cpi_Food_Beverages_PoP_x,cpi_health_YoY_x,vehicles_units_PoP_x,cpi_health_PoP_y,cpi_Food_Beverages_PoP_y,cpi_health_YoY_y,vehicles_units_PoP_y
date,,,,,,,,,,,,,,,,,,,,,
2017-04-09,39.6,7,4,65,5,5,23,42,12,0,...,0.0,27020.0,0.200965,0.135344,3.807947,-0.604067,0.200965,0.135344,3.807947,-0.604067
2017-04-16,44.8,14,1,46,3,3,18,17,15,1,...,54.0,13709.0,0.200965,0.135344,3.807947,-3.106219,0.200965,0.135344,3.807947,-3.106219
2017-04-23,45.4,16,2,46,9,13,35,47,22,1,...,59.0,12122.0,0.200965,0.135344,3.807947,23.771691,0.200965,0.135344,3.807947,23.771691
2017-04-30,40.5,8,9,145,20,25,80,87,31,2,...,46.0,11646.0,0.200965,0.135344,3.807947,83.975225,0.200965,0.135344,3.807947,83.975225
2017-05-07,45.0,9,3,116,14,22,67,67,33,3,...,0.0,11844.0,0.179426,-0.076923,4.293972,-8.615413,0.179426,-0.076923,4.293972,-8.615413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-03,23.5,7,9,132,5,6,24,85,21,53,...,59.0,10509.0,0.104749,-0.237573,39.017788,-6.999307,0.104749,-0.237573,39.017788,-6.999307
2019-03-10,26.4,6,8,138,1,2,12,65,19,59,...,20.0,10638.0,0.052156,-0.147601,11.102544,0.684189,0.052156,-0.147601,11.102544,0.684189
2019-03-17,28.9,9,19,123,2,4,32,80,19,60,...,0.0,11039.0,0.052156,-0.147601,11.102544,-8.437058,0.052156,-0.147601,11.102544,-8.437058


In [947]:
df_ludhiana_trainset.columns

Index(['tempmax', 'Outcome_EXPIRY_sum', 'Outcome_DAMA_sum',
       'Outcome_DISCHARGE_sum', 'SMOKING__sum', 'ALCOHOL_sum', 'DM_sum',
       'HTN_sum', 'PRIOR_CMP_sum', 'readmissions_sum', 'CKD_sum',
       'SEVERE_ANAEMIA_sum', 'ANAEMIA_sum', 'STABLE_ANGINA_sum', 'ACS_sum',
       'STEMI_sum', 'ATYPICAL_CHEST_PAIN_sum', 'HEART_FAILURE_sum',
       'HFREF_sum', 'HFNEF_sum', 'VALVULAR_sum', 'CAD_sum', 'total_admissions',
       'condition_rainy', 'TYPE_OF_ADMISSION_EMERGENCY_OPD_sum', 'temp', 'dew',
       'humidity', 'windspeed', 'winddir', 'sealevelpressure', 'PM10 AVG',
       'PM2.5 AVG', 'visibility', 'TLC_median', 'PLATELETS_median',
       'GLUCOSE_median', 'UREA_median', 'CREATININE_median', 'BNP_median',
       'solarenergy', 'uvindex', 'tempmin', 'sun_duration', 'cloudcover',
       'male_ratio', 'rural_ratio', 'EF_median', 'HB_mean',
       'DURATION_OF_STAY_median', 'duration_of_intensive_unit_stay_median',
       'average_age', 'search_HeroDMCHeartInstitute', 'search_chestpa

In [944]:
df_ludhiana_trainset.drop(['cpi_health_PoP', 'cpi_tobac_PoP',
       'cpi_transport_PoP', 'cpi_health_YoY', 'cpi_tobac_YoY',
       'cpi_transport_YoY', 'cpi_health_cub', 'cpi_total',
       'cpi_health_cub_PoP', 'cpi_tobac_cub_PoP'], axis=1, inplace=True)

In [948]:
df_ludhiana_trainset.to_csv(f'{path}final_dataset_HA_Ludhiana_2017-2019_FINAL.csv')


In [775]:
df_cpi_ludhiana.to_csv(f'{path}CPI_data_Ludhiana_2013-2023_processed_YoY.csv')

In [114]:
def make_datetime(df, date_column):
    # Assuming your column is named 'date_column' and it's in a DataFrame called 'df'

    df[date_column] = pd.to_datetime(df[date_column].str.split('T').str[0])

    df.set_index(date_column, inplace=True)
    return df
def filter_dataframe_by_timespan(df):
    start_date = pd.Timestamp('2022-10-27')
    end_date = pd.Timestamp('2023-06-30')

    filtered_df = df.loc[(df.index >= start_date) & (df.index <= end_date)]
    return filtered_df


In [115]:
df_pm10_na = make_datetime(df_pm10_na, 'date')
df_pm25_na = make_datetime(df_pm25_na, 'date')
df_lu_weather = make_datetime(df_lu_weather, 'datetime')
df_lu_history = make_datetime(df_lu_history, 'datetime')
df_na_weather = make_datetime(df_na_weather, 'datetime')
df_lu_poll = make_datetime(df_lu_poll, 'date')

df_lu_weather.index.name = 'date'  # Change the datetime index name to 'date'
df_na_weather.index.name = 'date'  # Change the datetime index name to 'date'
df_lu_history.index.name = 'date'

filtered_df_pm10_na = filter_dataframe_by_timespan(df_pm10_na)
filtered_df_pm25_na = filter_dataframe_by_timespan(df_pm25_na)
filtered_df_lu_weather = filter_dataframe_by_timespan(df_lu_weather)
filtered_df_na_weather = filter_dataframe_by_timespan(df_na_weather)
filtered_df_lu_poll = filter_dataframe_by_timespan(df_lu_poll)
filtered_df_lu_history = filter_dataframe_by_timespan(df_lu_history)

In [238]:
df_pm25_na_embassy

,pm25
date,
2022-01-02,37
2022-01-01,57
2021-12-31,47
2021-12-30,43
2021-12-29,42
...,...
2023-07-05,114
2023-07-04,92
2023-07-03,76


In [19]:
#df_pm25_na_embassy = df_pm25_na_embassy.sort_index(ascending=False)

# Convert the 'Year' column to datetime and set it as the index
df_pm25_na_embassy['date'] = pd.to_datetime(df_pm25_na_embassy['date'])
df_pm25_na_embassy.set_index('date', inplace=True)
df_pm25_na_embassy.drop('date', axis=1, inplace=True)

KeyError: 'Unnamed: 0'

In [21]:
df_pm25_na_embassy.rename(columns={' pm25':'PM2.5 AVG'}, inplace=True)


In [22]:
df_pm25_na_embassy

,PM2.5 AVG
date,
2022-01-02,37
2022-01-01,57
2021-12-31,47
2021-12-30,43
2021-12-29,42
...,...
2023-07-05,114
2023-07-04,92
2023-07-03,76


In [ ]:
def check_missing_dates(df):
    # Generate a complete date range from the minimum to maximum dates in the index
    date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')

    # Identify the missing dates by finding the set difference between 
    # the complete date range and the DataFrame index
    missing_dates = set(date_range) - set(df.index)

    # Print the missing dates
    print("Missing Dates:")
    for date in missing_dates:
        print(date)
        
check_missing_dates(df_lu_weather)
check_missing_dates(df_lu_history)
check_missing_dates(df_na_weather)
check_missing_dates(df_pm10_na)
check_missing_dates(df_pm25_na)

In [108]:
df_lu_history.isna().sum()

name                  0
tempmax               0
tempmin               0
temp                  0
feelslikemax          0
feelslikemin          0
feelslike             0
dew                   0
humidity              0
precip                0
precipprob            0
precipcover           0
preciptype          530
snow                  0
snowdepth             0
windgust            786
windspeed             0
winddir               0
sealevelpressure      0
cloudcover            0
visibility            0
solarradiation        0
solarenergy           0
uvindex               0
severerisk          786
sunrise               0
sunset                0
moonphase             0
conditions            0
description           0
icon                  0
stations              0
dtype: int64

In [87]:
df_lu_history.shape

(737, 32)

# Feature Engineering Weather Data

In [198]:
def create_sun_duration(df):
    df['sunrise'] = pd.to_datetime(df['sunrise'])
    df['sunset'] = pd.to_datetime(df['sunset'])

    # Calculate the duration of sun for each day in minutes
    df['sun_duration'] = (df['sunset'] - df['sunrise']).dt.total_seconds() / 60
    
    # Drop the 'sunrise' and 'sunset' columns
    df.drop(['sunrise', 'sunset'], axis=1, inplace=True)
    
    return df


def preprocess_weather(df):
    # new column
    df = create_sun_duration(df)
    
    # Create dummy variables with 0 and 1 values
    dummy_df = pd.get_dummies(df['conditions'], prefix='condition', dtype=int)

    # Join the original DataFrame with the dummy variables
    df = df.join(dummy_df)
    
    # change variable name seperators
    df.columns = (column.replace(" ", "_").replace("/", 
                  "_").replace("&", "_").replace("-",
                  "_").replace("(", "_").replace(")", "_")
                  for column in df.columns)
    
    
    # Rain is known to wash away pollutants so we aggregate those features
    df['condition_rainy'] = ((df['condition_Rain,_Overcast'] == 1) |
                         (df['condition_Rain,_Partially_cloudy'] == 1)).astype(int)
    
        

    df.drop(['condition_Partially_cloudy', 
     'condition_Rain,_Overcast',
       'condition_Rain,_Partially_cloudy'], axis=1, inplace=True)
    
    # drop uninforming columns
    df.drop(['precip', 'precipprob', 'conditions',
                    'precipcover', 'moonphase', 'feelslikemax',
                    'description', 'icon', 'feelslikemin',
                    'stations','snow', 'feelslike',
                    'snowdepth', 'name', 'preciptype'], axis=1, inplace=True)
    
    # drop columns that are not usable because of too many missing values
    df.drop(['windgust', 'severerisk'], axis=1, inplace=True)
    df['date'] = pd.to_datetime(df['datetime'])
    df.set_index('date', inplace=True)
    df.drop('datetime', axis=1, inplace=True)

    return df

#df_lu_weather = preprocess_weather(df_lu_weather)
#df_lu_history = preprocess_weather(df_lu_history)
#df_na_weather = preprocess_weather(df_na_weather)

In [200]:
df_na_weather.to_csv(f'{path}nairobi_weather_2022-01-01_to_2023-06-30_preprocessed.csv')
df_lu_weather.to_csv(f'{path}luidhiana_weather_2022-01-01_to_2023-06-30_preprocessed.csv')


In [655]:
df_na_weather = pd.read_csv(f'{path}nairobi_weather_2022-01-01_to_2023-06-30_preprocessed.csv')

In [165]:
 # Create dummy variables with 0 and 1 values
dummy_df = pd.get_dummies(df_na_weather['conditions'], prefix='condition', dtype=int)
# Join the original DataFrame with the dummy variables
df_na_weather = df_na_weather.join(dummy_df)

In [166]:
df_na_weather.columns

Index(['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
       'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
       'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
       'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations',
       'condition_Clear', 'condition_Overcast', 'condition_Partially cloudy',
       'condition_Rain, Overcast', 'condition_Rain, Partially cloudy'],
      dtype='object')

# Motor Vehicles Registered Ludhiana(Train Data)

In [ ]:
# get rid of metadata
df_lu_vehicles = df_lu_vehicles.iloc[26:-1]

# Convert the 'Year' column to datetime and set it as the index
df_lu_vehicles['date'] = pd.to_datetime(df_lu_vehicles['date'])
df_lu_vehicles.set_index('date', inplace=True)

# Rename the column with the 'Registered Motor Vehicles' data to a valid Python identifier
df_lu_vehicles.columns = ['Registered_Motor_Vehicles']
df_lu_vehicles = df_lu_vehicles['Registered_Motor_Vehicles'].astype(int)


In [67]:
# get rid of metadata
df_lu_ci = df_lu_ci.iloc[26:-1]

# Convert the 'Year' column to datetime and set it as the index
df_lu_ci['date'] = pd.to_datetime(df_lu_ci['Unnamed: 0'])
df_lu_ci.set_index('date', inplace=True)
df_lu_ci.drop('Unnamed: 0', axis=1, inplace=True)

# Rename the column with the 'Registered Motor Vehicles' data to a valid Python identifier
df_lu_ci.columns = ['Average_Consumer_Prices_Industrial_Workers_Ludhiana_Rice']
df_lu_ci = df_lu_ci['Average_Consumer_Prices_Industrial_Workers_Ludhiana_Rice'].astype(float)

In [119]:
df_lu_history.to_csv(f'{path}Ludhiana_historic_weather_preprocessed.csv')
df_lu_weather.to_csv(f'{path}Ludhiana_2022-2023_weather_preprocessed.csv')
df_na_weather.to_csv(f'{path}Nairobi_2022-2023_weather_preprocessed.csv')
#df_lu_vehicles_daily.to_csv(f'{path}Ludhiana_Vehicles_Sales.csv')

In [69]:
df_lu_ci_daily.to_csv(f'{path}AVG_ConsumerPriceIndex_Rice_Ludhiana.csv')

In [509]:
df_lu_vehicles = pd.read_csv(f'{path}Ludhiana_Vehicles_Sales.csv')
df_lu_vehicles['date'] = pd.to_datetime(df_lu_vehicles['date'])
df_lu_vehicles.set_index('date', inplace=True)

In [931]:
df_lu_vehicles

,Registered_Motor_Vehicles
date,
2016-01-03,11349
2016-01-10,11438
2016-01-17,10292
2016-01-24,13464
2016-01-31,12531
...,...
2023-06-04,8539
2023-06-11,7659
2023-06-18,7948


In [202]:
df_weekly.to_csv(f'{path}nairobi_registered_vehicles_2022-2023.csv')

In [ ]:
df_lu_ci

In [55]:
df_na_weekly = pd.read_csv(f'{path}Nairobi_Feature_Set_final_2022-2023.csv')


In [59]:
df_na_weekly

,tempmax,PM2.5 AVG,condition_rainy,tempmin,temp,dew,humidity,windspeed,winddir,sealevelpressure,visibility,solarenergy,uvindex,sun_duration,cloudcover,search_hospital,vehicles_units_PoP,cpi_health_PoP,cpi_health_YoY,cpi_Food_Beverages_PoP
date,,,,,,,,,,,,,,,,,,,,
2022-01-02,26.0,57.0,0,14.450000,20.500000,13.250000,66.200000,26.650000,62.200000,1020.050000,10.400000,23.550000,9.500000,731.591667,45.650000,25,1.018931,0.022516,0.187287,0.212250
2022-01-09,27.7,52.0,0,16.614286,20.757143,13.357143,66.328571,29.600000,65.571429,1020.214286,10.714286,24.914286,9.714286,731.459524,67.357143,32,1.018931,0.022516,0.187287,0.212250
2022-01-16,28.1,69.0,2,15.028571,20.471429,12.357143,63.928571,33.985714,56.471429,1019.571429,10.857143,26.214286,8.857143,731.197619,55.714286,0,1.018931,0.022516,0.187287,0.212250
2022-01-23,27.0,75.0,5,15.642857,19.857143,13.900000,71.528571,30.271429,92.585714,1019.942857,10.457143,18.542857,7.285714,730.852381,83.057143,22,1.018931,0.022516,0.187287,0.212250
2022-01-30,29.0,82.0,0,15.557143,21.057143,12.414286,60.742857,28.528571,66.228571,1018.828571,10.371429,25.014286,8.571429,730.447619,54.400000,40,1.018931,0.022516,0.187287,0.212250
2022-02-06,29.0,67.0,2,16.671429,21.214286,13.371429,63.628571,27.314286,70.300000,1018.814286,10.300000,17.971429,7.571429,729.990476,76.242857,79,-23.322228,0.014057,0.224438,0.208146
2022-02-13,29.0,83.0,0,16.614286,21.642857,12.142857,57.042857,26.671429,60.942857,1018.814286,10.385714,24.442857,8.857143,729.497619,61.671429,28,-23.322228,0.014057,0.224438,0.208146
2022-02-20,30.0,82.0,1,16.642857,22.528571,12.357143,56.442857,24.271429,70.528571,1017.800000,10.357143,24.614286,8.857143,728.983333,68.771429,43,-23.322228,0.014057,0.224438,0.208146
2022-02-27,29.0,88.0,4,15.857143,20.700000,13.957143,69.142857,32.028571,121.328571,1019.671429,10.142857,21.957143,8.000000,728.457143,75.671429,18,-23.322228,0.014057,0.224438,0.208146


In [57]:
df_na_weekly

,tempmax,PM2.5 AVG,condition_rainy,tempmin,temp,dew,humidity,windspeed,winddir,sealevelpressure,visibility,solarenergy,uvindex,sun_duration,cloudcover,search_hospital,vehicles_units_PoP,cpi_health_PoP,cpi_health_YoY,cpi_Food_Beverages_PoP
date,,,,,,,,,,,,,,,,,,,,
2022-01-02,26.0,57.0,0,14.450000,20.500000,13.250000,66.200000,26.650000,62.200000,1020.050000,10.400000,23.550000,9.500000,731.591667,45.650000,25,1.018931,0.022516,0.187287,0.212250
2022-01-09,27.7,52.0,0,16.614286,20.757143,13.357143,66.328571,29.600000,65.571429,1020.214286,10.714286,24.914286,9.714286,731.459524,67.357143,32,1.018931,0.022516,0.187287,0.212250
2022-01-16,28.1,69.0,2,15.028571,20.471429,12.357143,63.928571,33.985714,56.471429,1019.571429,10.857143,26.214286,8.857143,731.197619,55.714286,0,1.018931,0.022516,0.187287,0.212250
2022-01-23,27.0,75.0,5,15.642857,19.857143,13.900000,71.528571,30.271429,92.585714,1019.942857,10.457143,18.542857,7.285714,730.852381,83.057143,22,1.018931,0.022516,0.187287,0.212250
2022-01-30,29.0,82.0,0,15.557143,21.057143,12.414286,60.742857,28.528571,66.228571,1018.828571,10.371429,25.014286,8.571429,730.447619,54.400000,40,1.018931,0.022516,0.187287,0.212250
2022-02-06,29.0,67.0,2,16.671429,21.214286,13.371429,63.628571,27.314286,70.300000,1018.814286,10.300000,17.971429,7.571429,729.990476,76.242857,79,-23.322228,0.014057,0.224438,0.208146
2022-02-13,29.0,83.0,0,16.614286,21.642857,12.142857,57.042857,26.671429,60.942857,1018.814286,10.385714,24.442857,8.857143,729.497619,61.671429,28,-23.322228,0.014057,0.224438,0.208146
2022-02-20,30.0,82.0,1,16.642857,22.528571,12.357143,56.442857,24.271429,70.528571,1017.800000,10.357143,24.614286,8.857143,728.983333,68.771429,43,-23.322228,0.014057,0.224438,0.208146
2022-02-27,29.0,88.0,4,15.857143,20.700000,13.957143,69.142857,32.028571,121.328571,1019.671429,10.142857,21.957143,8.000000,728.457143,75.671429,18,-23.322228,0.014057,0.224438,0.208146


In [56]:
df_na_weekly['date'] = pd.to_datetime(df_na_weekly['Unnamed: 0'])
df_na_weekly.set_index('date', inplace=True)
df_na_weekly.drop('Unnamed: 0', axis=1, inplace=True)

In [50]:
from sklearn.impute import KNNImputer

# Create imputer
imputer = KNNImputer(n_neighbors=5)

# Fit on the data and transform the data
df_na_weekly_imputed = pd.DataFrame(df_na_weekly, columns=df_na_weekly.columns, index=df_na_weekly.index)


df_na_weekly_imputed


,tempmax,PM2.5 AVG,condition_rainy,tempmin,temp,dew,humidity,windspeed,winddir,sealevelpressure,visibility,solarenergy,uvindex,sun_duration,cloudcover,search_hospital,vehicles_units_PoP,cpi_health_PoP,cpi_health_YoY,cpi_Food_Beverages_PoP
date,,,,,,,,,,,,,,,,,,,,
2022-01-02,26.0,57.0,0,14.450000,20.500000,13.250000,66.200000,26.650000,62.200000,1020.050000,10.400000,23.550000,9.500000,731.591667,45.650000,25,1.018931,0.022516,0.187287,0.212250
2022-01-09,27.7,52.0,0,16.614286,20.757143,13.357143,66.328571,29.600000,65.571429,1020.214286,10.714286,24.914286,9.714286,731.459524,67.357143,32,1.018931,0.022516,0.187287,0.212250
2022-01-16,28.1,69.0,2,15.028571,20.471429,12.357143,63.928571,33.985714,56.471429,1019.571429,10.857143,26.214286,8.857143,731.197619,55.714286,0,1.018931,0.022516,0.187287,0.212250
2022-01-23,27.0,75.0,5,15.642857,19.857143,13.900000,71.528571,30.271429,92.585714,1019.942857,10.457143,18.542857,7.285714,730.852381,83.057143,22,1.018931,0.022516,0.187287,0.212250
2022-01-30,29.0,82.0,0,15.557143,21.057143,12.414286,60.742857,28.528571,66.228571,1018.828571,10.371429,25.014286,8.571429,730.447619,54.400000,40,1.018931,0.022516,0.187287,0.212250
2022-02-06,29.0,67.0,2,16.671429,21.214286,13.371429,63.628571,27.314286,70.300000,1018.814286,10.300000,17.971429,7.571429,729.990476,76.242857,79,-23.322228,0.014057,0.224438,0.208146
2022-02-13,29.0,83.0,0,16.614286,21.642857,12.142857,57.042857,26.671429,60.942857,1018.814286,10.385714,24.442857,8.857143,729.497619,61.671429,28,-23.322228,0.014057,0.224438,0.208146
2022-02-20,30.0,82.0,1,16.642857,22.528571,12.357143,56.442857,24.271429,70.528571,1017.800000,10.357143,24.614286,8.857143,728.983333,68.771429,43,-23.322228,0.014057,0.224438,0.208146
2022-02-27,29.0,88.0,4,15.857143,20.700000,13.957143,69.142857,32.028571,121.328571,1019.671429,10.142857,21.957143,8.000000,728.457143,75.671429,18,-23.322228,0.014057,0.224438,0.208146


In [54]:
df_na_weekly.loc['2022-08-14']

tempmax                         NaN
PM2.5 AVG                       NaN
condition_rainy            0.000000
tempmin                         NaN
temp                            NaN
dew                             NaN
humidity                        NaN
windspeed                       NaN
winddir                         NaN
sealevelpressure                NaN
visibility                      NaN
solarenergy                     NaN
uvindex                         NaN
sun_duration                    NaN
cloudcover                      NaN
search_hospital           48.000000
vehicles_units_PoP        -6.821105
cpi_health_PoP             0.075000
cpi_health_YoY             0.375000
cpi_Food_Beverages_PoP     0.125000
Name: 2022-08-14 00:00:00, dtype: float64

# Merge DataFrame Nairobi 2022

In [4]:
"""
1. ) health POP distributed
2. ) food and beverages pop distributed
3. ) health YoY distributed
4. ) registered motor vehicles PoP weekly(original weekly data)
"""

def distribute_monthly_to_weekly(df, col):
    # Create a new DataFrame that includes all Sundays in the date range of our data
    all_sundays = pd.date_range(df.index.min(), df.index.max(), freq='W-SUN')
    df_weekly = pd.DataFrame(index=all_sundays)

    # For each month in the original data, distribute the monthly value equally among the corresponding Sundays
    for month in df.index:
        # Identify the Sundays in this month
        sundays_in_month = df_weekly.loc[month:month + pd.DateOffset(months=1)].index
        # Distribute the monthly value equally among these Sundays
        df_weekly.loc[sundays_in_month, col] = df.loc[month, col] / len(sundays_in_month)

    # Merge the new DataFrame with the original one
    # df = df.merge(df_weekly, how='outer', left_index=True, right_index=True, suffixes=('', '_weekly'))
    

    return df_weekly


cpi_health_pop_na = pd.read_csv(f'{path}CPI Feb2019100 Health_PoP_2022-2023.csv')
cpi_FaB_pop_na = pd.read_csv(f'{path}CPI Feb2019100 Food and Non Alcoholic Beverages_2022-2023_PoP.csv')
cpi_health_yoy_na = pd.read_csv(f'{path}CPI Feb2019100 Health_YoY_2022-2023.csv')
vehicles_pop_na = pd.read_csv(f'{path}Road Transport No of Motor Vehicles Newly Registered_2022-2023_kenya.csv')

cpi_health_pop_na['date'] = pd.to_datetime(cpi_health_pop_na['date'], format='%b %y')
cpi_health_pop_na.set_index('date', inplace=True)
cpi_health_pop_na = distribute_monthly_to_weekly(cpi_health_pop_na, ' cpi_health_PoP ')

cpi_FaB_pop_na['date'] = pd.to_datetime(cpi_FaB_pop_na['date'], format='%b %y')
cpi_FaB_pop_na.set_index('date', inplace=True)
cpi_FaB_pop_na = distribute_monthly_to_weekly(cpi_FaB_pop_na, ' cpi_FaB_pop ')

cpi_health_yoy_na['date'] = pd.to_datetime(cpi_health_yoy_na['date'], format='%b %y')
cpi_health_yoy_na.set_index('date', inplace=True)
cpi_health_yoy_na = distribute_monthly_to_weekly(cpi_health_yoy_na, ' cpi_health_yoy ')

vehicles_pop_na['date'] = pd.to_datetime(vehicles_pop_na['date'], format='%b %y')
vehicles_pop_na.set_index('date', inplace=True)
vehicles_pop_na = distribute_monthly_to_weekly(vehicles_pop_na, 'vehicles_pop')


vehicles_pop_na
cpi_health_pop_na
cpi_FaB_pop_na
cpi_health_yoy_na

/var/folders/_t/kmv70h2x57sbxhd8w5ybzqwr0000gn/T/ipykernel_2951/2161351141.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  df_weekly.loc[sundays_in_month, col] = df.loc[month, col] / len(sundays_in_month)
/var/folders/_t/kmv70h2x57sbxhd8w5ybzqwr0000gn/T/ipykernel_2951/2161351141.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  df_weekly.loc[sundays_in_month, col] = df.loc[month, col] / len(sundays_in_month)
/var/folders/_t/kmv70h2x57sbxhd8w5ybzqwr0000gn/T/ipykernel_2951/2161351141.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  df_weekly.loc[sundays_in_month, col] = df.loc[month, col] / len(sundays_in_month)
/var/folders/_t/kmv70h2x57sbxhd8w5ybzqwr0000gn/T/ipykernel_2951/2161351141.py:18: RuntimeWarning: divide by zero encountered in double_scalars
  df_weekly.loc[sundays_in_month, col] = df.loc[month, col] / len(sundays_in_month)


,cpi_health_yoy
2010-02-07,1.220000
2010-02-14,1.220000
2010-02-21,1.220000
2010-02-28,1.220000
2010-03-07,0.919699
...,...
2023-01-01,0.420000
2023-01-08,0.420000
2023-01-15,0.420000
2023-01-22,0.420000


In [5]:
print(vehicles_pop_na.index[0].weekday())
print(cpi_health_pop_na.index[0].weekday())
print(cpi_FaB_pop_na.index[0].weekday())
print(cpi_health_yoy_na.index[0].weekday())


6
6
6
6


In [6]:

df_na_merge = vehicles_pop_na.merge(cpi_health_pop_na, left_index=True, right_index=True)
df_na_merge = df_na_merge.merge(cpi_health_yoy_na, left_index=True, right_index=True)
df_na_merge = df_na_merge.merge(cpi_FaB_pop_na, left_index=True, right_index=True)


In [9]:
df_aga = pd.read_csv(f'{path}search_agakhanhospital.csv')
df_aga = df_aga.iloc[1:]
df_aga['search_agakhanhospital'] = df_aga['Kategorie: Alle Kategorien']
df_aga.drop('Kategorie: Alle Kategorien', axis=1, inplace=True)
df_aga.index = pd.to_datetime(df_aga.index)
df_aga['search_agakhanhospital'] = df_aga['search_agakhanhospital'].astype(int)
df_aga['search_agakhanhospital'] = df_aga.resample('W-SUN').last()

In [23]:
df_final = df_na_weather.merge(df_pm25_na_embassy, left_index=True, right_index=True)

# preprocess daily weather and pollution data
max_cols = ['tempmax', 'PM2.5 AVG']
min_cols = ['tempmin']
mean_cols = ['temp', 'dew', 'humidity', 'windspeed', 'winddir', 'sealevelpressure', 'visibility',
             'solarenergy', 'tempmin', 'uvindex',
             'sun_duration', 'cloudcover', ]
sum_cols = ['condition_rainy']
# Aggregate the dataframe on a weekly basis
df_na_weekly = pd.DataFrame()
df_na_weekly[max_cols] = df_final[max_cols].resample('W-SUN').max()
df_na_weekly[sum_cols] = df_final[sum_cols].resample('W-SUN').sum()
df_na_weekly[min_cols] = df_final[min_cols].resample('W-SUN').min()
df_na_weekly[mean_cols] = df_final[mean_cols].resample('W-SUN').mean()


KeyError: "None of [Index(['condition_rainy'], dtype='object')] are in the [columns]"

In [1086]:
df_na_weekly = df_na_weekly.merge(df_aga, left_index=True, right_index=True)



In [1088]:
df_na_weekly = df_na_weekly.merge(df_na_merge, left_index=True, right_index=True)


In [67]:
df_na_weather.loc['2022-08-07':'2022-08-14']

,tempmax,tempmin,temp,dew,humidity,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,sun_duration,condition_Clear,condition_Overcast,condition_Partially_cloudy,"condition_Rain,_Overcast","condition_Rain,_Partially_cloudy"
date,,,,,,,,,,,,,,,,,,,
2022-08-07,22.0,11.0,16.3,11.0,72.7,26.9,144.3,1021.7,89.9,10.2,167.5,14.5,7,723.933333,0,0,1,0,0
2022-08-08,21.9,13.1,16.9,9.5,63.5,21.9,146.1,1021.8,87.4,10.2,161.2,13.9,7,723.983333,0,0,1,0,0
2022-08-09,23.1,12.0,17.1,10.0,65.1,17.1,113.6,1022.0,87.5,10.3,186.1,16.1,10,724.033333,0,0,1,0,0
2022-08-10,25.0,13.0,18.1,10.3,62.1,26.5,131.7,1022.0,86.7,10.2,219.9,19.0,9,724.066667,0,0,1,0,0
2022-08-11,21.0,13.1,16.9,11.5,72.5,18.6,137.6,1022.3,89.5,10.1,122.8,10.6,7,724.116667,0,0,1,0,0
2022-08-12,21.0,13.0,16.7,9.6,64.6,20.5,151.1,1022.2,88.9,10.0,134.5,11.7,7,724.166667,0,0,0,0,1
2022-08-13,21.0,13.0,16.7,12.1,75.4,22.3,150.3,1021.2,89.9,9.9,127.5,11.2,6,724.216667,0,0,0,0,1
2022-08-14,21.9,13.9,17.1,12.3,75.0,20.7,91.6,1021.1,88.9,10.2,208.1,18.0,10,724.266667,0,0,1,0,0


In [68]:
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['tempmax'].max())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['PM2.5 AVG'].max())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['condition_rainy'].sum())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['tempmin'].min())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['dew'].mean())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['humidity'].mean())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['windspeed'].mean())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['winddir'].mean())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['sealevelpressure'].mean())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['visibility'].mean())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['solarenergy'].mean())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['uvindex'].mean())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['sun_duration'].mean())
print(df_na_weekly.loc['2022-08-07':'2022-08-14']['cloudcover'].mean())

19.1
92.0
2
14.366666666666667
12.633333333333333
80.13333333333334
15.366666666666667
118.93333333333334
1022.8666666666668
10.1
9.066666666666668
4.666666666666667
723.7166666666667
90.26666666666668


In [1090]:
df_na_weekly = df_na_weekly.rename(columns={
    'search_agakhanhospital': 'search_hospital',
    'vehicles_pop': 'vehicles_units_PoP',
    ' cpi_health_yoy ': 'cpi_health_YoY',
    ' cpi_FaB_pop ': 'cpi_Food_Beverages_PoP',
    ' cpi_health_PoP ': 'cpi_health_PoP',
})

In [37]:
df_na_weekly = pd.read_csv(f'{path}Nairobi_Feature_Set_final_2022-2023.csv')

In [41]:
df_na_weekly

,Unnamed: 0,tempmax,PM2.5 AVG,condition_rainy,tempmin,temp,dew,humidity,windspeed,winddir,...,solarenergy,uvindex,sun_duration,cloudcover,search_hospital,vehicles_units_PoP,cpi_health_PoP,cpi_health_YoY,cpi_Food_Beverages_PoP,date
0,2022-01-02,26.0,57.0,0,14.450000,20.500000,13.250000,66.200000,26.650000,62.200000,...,23.550000,9.500000,731.591667,45.650000,25,1.018931,0.022516,0.187287,0.212250,2022-01-02
1,2022-01-09,27.7,52.0,0,16.614286,20.757143,13.357143,66.328571,29.600000,65.571429,...,24.914286,9.714286,731.459524,67.357143,32,1.018931,0.022516,0.187287,0.212250,2022-01-09
2,2022-01-16,28.1,69.0,2,15.028571,20.471429,12.357143,63.928571,33.985714,56.471429,...,26.214286,8.857143,731.197619,55.714286,0,1.018931,0.022516,0.187287,0.212250,2022-01-16
3,2022-01-23,27.0,75.0,5,15.642857,19.857143,13.900000,71.528571,30.271429,92.585714,...,18.542857,7.285714,730.852381,83.057143,22,1.018931,0.022516,0.187287,0.212250,2022-01-23
4,2022-01-30,29.0,82.0,0,15.557143,21.057143,12.414286,60.742857,28.528571,66.228571,...,25.014286,8.571429,730.447619,54.400000,40,1.018931,0.022516,0.187287,0.212250,2022-01-30
5,2022-02-06,29.0,67.0,2,16.671429,21.214286,13.371429,63.628571,27.314286,70.300000,...,17.971429,7.571429,729.990476,76.242857,79,-23.322228,0.014057,0.224438,0.208146,2022-02-06
6,2022-02-13,29.0,83.0,0,16.614286,21.642857,12.142857,57.042857,26.671429,60.942857,...,24.442857,8.857143,729.497619,61.671429,28,-23.322228,0.014057,0.224438,0.208146,2022-02-13
7,2022-02-20,30.0,82.0,1,16.642857,22.528571,12.357143,56.442857,24.271429,70.528571,...,24.614286,8.857143,728.983333,68.771429,43,-23.322228,0.014057,0.224438,0.208146,2022-02-20
8,2022-02-27,29.0,88.0,4,15.857143,20.700000,13.957143,69.142857,32.028571,121.328571,...,21.957143,8.000000,728.457143,75.671429,18,-23.322228,0.014057,0.224438,0.208146,2022-02-27
9,2022-03-06,29.1,76.0,0,15.857143,22.314286,10.585714,51.028571,30.528571,66.328571,...,26.385714,9.857143,727.928571,44.942857,0,160.496424,0.044488,0.266938,0.371941,2022-03-06


In [40]:
df_na_weekly.i = pd.to_datetime(df_na_weekly['Unnamed: 0'])

In [33]:
df_na_weekly.index = df_na_weekly['date']
df_na_weekly.drop('date', inplace=True)

In [36]:
df_na_weekly.drop('Unnamed: 0', inplace=True)
df_na_weekly

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
from sklearn.impute import KNNImputer

# Create imputer
imputer = KNNImputer(n_neighbors=5)

# Fit on the data and transform the data
nairobi_imputed = imputer.fit_transform(nairobi_df)

# Convert back to dataframe
nairobi_imputed_df = pd.DataFrame(nairobi_imputed, columns=nairobi_df.columns, index=nairobi_df.index)

nairobi_imputed_df.tail(10)


In [1058]:
print(df_aga.index[0].weekday())

6


# Merge Dataframe Luidhiana 2022

In [578]:
df_lu_weather = pd.read_csv(f'{path}luidhiana_weather_2022-01-01_to_2023-06-30_preprocessed.csv')
df_lu_weather['date'] = pd.to_datetime(df_lu_weather['date'])
df_lu_weather.set_index('date', inplace=True)

In [579]:
df_hero = pd.read_csv(f'{path}search_HeroHeartHospital_2022-23.csv')
df_hero = df_hero.iloc[1:]
df_hero['search_HeroDMCHeartInstitute'] = df_hero['Kategorie: Alle Kategorien']
df_hero.drop('Kategorie: Alle Kategorien', axis=1, inplace=True)
df_hero.index = pd.to_datetime(df_hero.index)
df_hero['search_HeroDMCHeartInstitute'] = df_hero['search_HeroDMCHeartInstitute'].astype(int)
df_hero['search_HeroDMCHeartInstitute'] = df_hero.resample('W-SUN').last()


In [586]:
df_lu_poll = pd.read_csv(f'{path}Air Pollutant at 400pm PM25 Punjab Ludhiana Average.csv')
df_lu_poll = preprocess_cec(df_lu_poll, 'PM2.5 AVG')

In [1083]:
df_final_lu = df_lu_weather.merge(df_lu_poll, left_index=True, right_index=True)

# preprocess daily weather and pollution data
max_cols = ['tempmax', 'PM2.5 AVG']
min_cols = ['tempmin']
mean_cols = ['temp', 'dew', 'humidity', 'windspeed', 'winddir', 'sealevelpressure', 'visibility',
             'solarenergy', 'tempmin', 'uvindex',
             'sun_duration', 'cloudcover', ]
sum_cols = ['condition_rainy']
# Aggregate the dataframe on a weekly basis
df_weekly = pd.DataFrame()
df_weekly[max_cols] = df_final_lu[max_cols].resample('W-SUN').max()
df_weekly[min_cols] = df_final_lu[min_cols].resample('W-SUN').min()
df_weekly[sum_cols] = df_final_lu[sum_cols].resample('W-SUN').sum()
df_weekly[mean_cols] = df_final_lu[mean_cols].resample('W-SUN').mean()


In [557]:
# shift from monday to sunday
df_india_alc_ci_distributed.index = df_india_alc_ci_distributed.index - pd.DateOffset(days=1)
df_india_food_ci_distributed.index = df_india_food_ci_distributed.index - pd.DateOffset(days=1)

#shift from friday to sunday
df_lu_vehicles.index = df_lu_vehicles.index - pd.DateOffset(days=-2)

In [ ]:
print(df_india_alc_ci_distributed.index[0].weekday())
print(df_india_food_ci_distributed.index[0].weekday())
print(df_lu_vehicles.index[0].weekday())
print(df_hero.index[0].weekday())
print(df_weekly.index[0].weekday())

In [1079]:
df_lu_weekly = df_weekly.merge(df_hero, left_index=True, right_index=True)
#df_lu_weekly = df_lu_weekly.merge(df_india_alc_ci_distributed, left_index=True, right_index=True)
#df_lu_weekly = df_lu_weekly.merge(df_india_food_ci_distributed, left_index=True, right_index=True)
df_lu_weekly = df_lu_weekly.merge(df_lu_vehicles, left_index=True, right_index=True)
df_lu_weekly = df_lu_weekly.merge(df_cpi_ludhiana, left_index=True, right_index=True)

In [665]:
df_lu_weekly = df_lu_weekly.copy()
df_lu_weekly = df_lu_weekly.loc[:'2022-06-19']

In [1080]:
df_lu_weekly = df_lu_weekly.rename(columns={
    'search_HeroDMCHeartInstitute': 'search_hospital',
    'vehicles_pop': 'vehicles_units_PoP',
    'cpi_health_yoy': 'cpi_health_YoY',
    'cpi_FaB_pop': 'cpi_Food_Beverages_PoP',
})

In [1076]:
df_lu_weekly.to_csv(f'{path}Ludhiana_Feature_Set_2022-2023_final.csv')

In [950]:
df_lu_weekly.drop(['cpi_health', 'cpi_tobac', 'cpi_transport'], axis=1, inplace=True)

In [951]:
df_lu_weekly = df_lu_weekly.merge(merged_df, left_index=True, right_index=True)

In [954]:
df_lu_weekly.drop(['Registered_Motor_Vehicles'], axis=1, inplace=True)

In [1115]:
df_trainset_copy = df_ludhiana_trainset.copy()
df_trainset_copy['date'] = pd.to_datetime(df_trainset_copy['Unnamed: 0'])
df_trainset_copy.set_index('date', inplace=True)
df_trainset_copy.drop('Unnamed: 0', axis=1, inplace=True)
df_trainset_copy['uvindex'] = df_lu_weekly['uvindex']

In [1116]:
df_trainset_copy

,tempmax,PM2.5 AVG,tempmin,condition_rainy,temp,dew,humidity,windspeed,winddir,sealevelpressure,visibility,solarenergy,sun_duration,cloudcover,search_hospital,cpi_health_PoP,cpi_Food_Beverages_PoP,cpi_health_YoY,vehicles_units_PoP,uvindex
date,,,,,,,,,,,,,,,,,,,,
2022-01-02,20.4,150.0,6.300000,0,12.150000,1.150000,48.950000,10.250000,328.600000,1021.150000,2.300000,10.600000,611.208333,60.000000,36,0.071386,0.230863,5.701754,-14.343691,5.000000
2022-01-09,20.7,126.0,9.628571,5,13.242857,9.100000,78.457143,18.942857,223.257143,1016.842857,2.285714,7.585714,613.954762,65.714286,0,0.177830,-0.615616,6.054931,-8.202547,3.714286
2022-01-16,18.5,140.0,7.057143,1,11.157143,6.885714,76.471429,9.528571,318.414286,1018.828571,19.671429,12.471429,619.716667,33.171429,70,0.177830,-0.615616,6.054931,-8.016129,5.857143
2022-01-23,18.9,191.0,8.557143,5,11.642857,7.871429,78.957143,18.142857,192.500000,1014.328571,19.414286,9.471429,627.245238,67.385714,45,0.177830,-0.615616,6.054931,-8.802385,5.285714
2022-01-30,20.8,128.0,6.528571,3,10.914286,5.828571,72.828571,13.328571,293.500000,1015.900000,20.471429,13.785714,636.273810,12.314286,32,0.177830,-0.615616,6.054931,-6.094982,6.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-09,35.0,104.0,17.257143,1,23.757143,7.014286,39.371429,16.042857,333.471429,1011.742857,20.685714,23.785714,758.538095,21.900000,46,0.082554,0.310284,6.050029,-15.877172,8.857143
2023-04-16,41.0,137.0,22.571429,0,30.128571,2.857143,19.542857,17.957143,286.900000,1008.871429,20.642857,25.328571,771.314286,25.271429,66,0.082554,0.310284,6.050029,-15.877172,9.571429
2023-04-23,41.8,162.0,21.457143,5,27.842857,7.914286,33.000000,24.228571,102.614286,1005.528571,20.485714,23.214286,783.707143,43.542857,86,0.082554,0.310284,6.050029,-15.877172,9.428571


In [1117]:
df_trainset_copy.to_csv(f'{path}Ludhiana_Feature_Set_2022-2023_final.csv')
